#📌 Extracão

In [9]:
# Nome do arquivo: analise_churn_completa.py
# Descrição: Script definitivo que extrai dados de uma URL do GitHub, trata a estrutura aninhada e realiza a análise completa.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import requests # Biblioteca para fazer requisições HTTP

def extrair_dados_url(url: str) -> pd.DataFrame:
    """
    Fase de Extração: Carrega e achata os dados da URL do JSON fornecida.
    """
    print(f"Iniciando a Fase de Extração da URL: {url}")
    try:
        # Fazer a requisição HTTP para obter o conteúdo do JSON
        response = requests.get(url)
        response.raise_for_status()  # Lança um erro para status ruins (4xx ou 5xx)

        # Carregar o conteúdo de texto da resposta como JSON
        dados = response.json()

        # Usar json_normalize para achatar a estrutura aninhada
        df = pd.json_normalize(dados)
        print("Extração e achatamento de dados da URL concluídos com sucesso.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição HTTP: {e}")
        return None
    except Exception as e:
        print(f"Erro ao processar dados da URL: {e}")
        return None



#🔧 Transformação

In [10]:
def transformar_dados(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Fase de Transformação: Limpa, enriquece, traduz e codifica os dados.
    """
    print("\nIniciando a Fase de Transformação...")

    df_transformado = df.copy()

    # 1. Tratamento da coluna de cobranças totais
    df_transformado['account.Charges.Total'] = pd.to_numeric(df_transformado['account.Charges.Total'], errors='coerce')
    df_transformado['account.Charges.Total'].fillna(0, inplace=True)
    print("- Coluna de Cobranças Totais corrigida.")

    # 2. Engenharia de Recursos
    df_transformado['Fatura_Diaria'] = df_transformado['account.Charges.Monthly'] / 30
    print("- Coluna 'Fatura_Diaria' criada.")

    # 3. Tradução das Colunas
    mapa_colunas = {
        'customerID': 'ID_Cliente', 'Churn': 'Evasao',
        'customer.gender': 'Genero', 'customer.SeniorCitizen': 'Idoso',
        'customer.Partner': 'Parceiro', 'customer.Dependents': 'Dependentes',
        'customer.tenure': 'Meses_Contrato',
        'phone.PhoneService': 'Servico_Telefonico', 'phone.MultipleLines': 'Multiplas_Linhas',
        'internet.InternetService': 'Servico_Internet', 'internet.OnlineSecurity': 'Seguranca_Online',
        'internet.OnlineBackup': 'Backup_Online', 'internet.DeviceProtection': 'Protecao_Dispositivo',
        'internet.TechSupport': 'Suporte_Tecnico', 'internet.StreamingTV': 'Streaming_TV',
        'internet.StreamingMovies': 'Streaming_Filmes',
        'account.Contract': 'Contrato', 'account.PaperlessBilling': 'Fatura_Digital',
        'account.PaymentMethod': 'Metodo_Pagamento',
        'account.Charges.Monthly': 'Fatura_Mensal', 'account.Charges.Total': 'Fatura_Total'
    }
    df_transformado.rename(columns=mapa_colunas, inplace=True)
    print("- Nomes das colunas traduzidos.")

    # 4. Tradução dos Valores
    mapa_valores = {
        'Yes': 'Sim', 'No': 'Nao', 'No internet service': 'Sem servico de internet',
        'No phone service': 'Sem servico de telefone', 'Fiber optic': 'Fibra Otica',
        'Month-to-month': 'Mensal', 'One year': 'Um Ano', 'Two year': 'Dois Anos',
        'Electronic check': 'Cheque Eletronico', 'Mailed check': 'Cheque Papel',
        'Bank transfer (automatic)': 'Transferencia Bancaria',
        'Credit card (automatic)': 'Cartao de Credito'
    }
    df_transformado.replace(mapa_valores, inplace=True)
    print("- Valores categóricos traduzidos.")

    df_para_graficos = df_transformado.copy()
    print("- DataFrame para visualização criado.")

    # 5. Conversão Numérica
    df_transformado['Genero'] = df_transformado['Genero'].map({'Male': 1, 'Female': 0})
    colunas_binarias = ['Parceiro', 'Dependentes', 'Servico_Telefonico', 'Fatura_Digital', 'Evasao']
    for col in colunas_binarias:
        df_transformado[col] = df_transformado[col].map({'Sim': 1, 'Nao': 0})

    df_final_ml = pd.get_dummies(df_transformado, columns=[
        'Multiplas_Linhas', 'Servico_Internet', 'Seguranca_Online', 'Backup_Online',
        'Protecao_Dispositivo', 'Suporte_Tecnico', 'Streaming_TV', 'Streaming_Filmes',
        'Contrato', 'Metodo_Pagamento'
    ], drop_first=True)

    df_final_ml.drop('ID_Cliente', axis=1, inplace=True, errors='ignore')
    print("- Dados convertidos para formato numérico (pronto para ML).")

    print("Transformação de dados concluída.")
    return df_final_ml, df_para_graficos

#📊 Carga e análise

In [13]:
def analisar_dados(df_ml: pd.DataFrame, df_graficos: pd.DataFrame):
    """Fase de Análise: Gera estatísticas descritivas e visualizações."""
    print("\nIniciando a Fase de Análise...")

    if not os.path.exists('graficos_churn'):
        os.makedirs('graficos_churn')

    print("\n--- Estatísticas Descritivas ---")
    print(df_ml[['Idoso', 'Meses_Contrato', 'Fatura_Mensal', 'Fatura_Total', 'Evasao']].describe().T)

    sns.set_style("whitegrid")

    # Gráfico 1: Distribuição Geral do Churn (LINHA CORRIGIDA E COMPLETA)
    plt.figure(figsize=(8, 6))
    # Obter as contagens de valores e seus rótulos diretamente
    churn_counts = df_graficos['Evasao'].value_counts()
    # Usar o índice das contagens como rótulos para o gráfico de pizza
    churn_counts.plot(kind='pie', autopct='%1.1f%%', colors=['#4CAF50', '#F44336'][:len(churn_counts.index)], labels=churn_counts.index, textprops={'fontsize': 12})
    plt.title('Distribuição Geral de Evasão (Churn)', fontsize=16)
    plt.ylabel('')
    plt.savefig('graficos_churn/1_distribuicao_churn.png')
    plt.close()
    print("\n- Gráfico '1_distribuicao_churn.png' salvo.")

    # Gráfico 2: Evasão por Tipo de Contrato
    plt.figure(figsize=(10, 6))
    sns.countplot(x='Contrato', hue='Evasao', data=df_graficos, palette=['#4CAF50', '#F44336'], order=['Mensal', 'Um Ano', 'Dois Anos'])
    plt.title('Evasão por Tipo de Contrato', fontsize=16)
    plt.xlabel('Tipo de Contrato', fontsize=12)
    plt.ylabel('Contagem de Clientes', fontsize=12)
    plt.legend(title='Evasão')
    plt.savefig('graficos_churn/2_churn_por_contrato.png')
    plt.close()
    print("- Gráfico '2_churn_por_contrato.png' salvo.")

    print("\nAnálise concluída. Gráficos salvos na pasta 'graficos_churn'.")


if __name__ == "__main__":
    API_URL = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"
    dados_brutos_achatados = extrair_dados_url(API_URL)

    if dados_brutos_achatados is not None:
        df_modelo_final, df_para_visualizacao = transformar_dados(dados_brutos_achatados)
        analisar_dados(df_modelo_final, df_para_visualizacao)
        print("\nProcesso completo finalizado com sucesso!")

Iniciando a Fase de Extração da URL: https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json
Extração e achatamento de dados da URL concluídos com sucesso.

Iniciando a Fase de Transformação...
- Coluna de Cobranças Totais corrigida.
- Coluna 'Fatura_Diaria' criada.
- Nomes das colunas traduzidos.
- Valores categóricos traduzidos.
- DataFrame para visualização criado.
- Dados convertidos para formato numérico (pronto para ML).
Transformação de dados concluída.

Iniciando a Fase de Análise...

--- Estatísticas Descritivas ---


/tmp/ipython-input-10-2947134589.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_transformado['account.Charges.Total'].fillna(0, inplace=True)


                 count         mean          std    min      25%     50%  \
Idoso           7267.0     0.162653     0.369074   0.00    0.000     0.0   
Meses_Contrato  7267.0    32.346498    24.571773   0.00    9.000    29.0   
Fatura_Mensal   7267.0    64.720098    30.129572  18.25   35.425    70.3   
Fatura_Total    7267.0  2277.182035  2268.648587   0.00  396.200  1389.2   
Evasao          7043.0     0.265370     0.441561   0.00    0.000     0.0   

                     75%      max  
Idoso              0.000     1.00  
Meses_Contrato    55.000    72.00  
Fatura_Mensal     89.875   118.75  
Fatura_Total    3778.525  8684.80  
Evasao             1.000     1.00  

- Gráfico '1_distribuicao_churn.png' salvo.
- Gráfico '2_churn_por_contrato.png' salvo.

Análise concluída. Gráficos salvos na pasta 'graficos_churn'.

Processo completo finalizado com sucesso!


/tmp/ipython-input-13-438452467.py:27: UserWarning: 
The palette list has fewer values (2) than needed (3) and will cycle, which may produce an uninterpretable plot.
  sns.countplot(x='Contrato', hue='Evasao', data=df_graficos, palette=['#4CAF50', '#F44336'], order=['Mensal', 'Um Ano', 'Dois Anos'])


#📄Relatorio Final

In [ ]:
'''No competitivo setor de telecomunicações, a retenção de clientes é um pilar fundamental para a sustentabilidade e o crescimento do negócio.
A perda de um cliente, conhecida como churn ou evasão,não só impacta diretamente a receita, mas também acarreta custos significativamente
mais altos para adquirir um novo cliente do que para manter um existente. Com este desafio em mente, foi realizada uma análise aprofundada
sobre os dados da Telecom X, com o objetivo claro de dissecar os fatores que levam um cliente a cancelar seu serviço. A meta final é
transformar esses dados brutos em insights acionáveis e recomendações estratégicas que possam fortalecer a lealdade do cliente e reduzir
a taxa de evasão.

O ponto de partida foi um conjunto de dados em formato JSON, extraído de uma API, que continha uma rica variedade de informações sobre os
clientes, desde dados demográficos até os detalhes dos serviços contratados. A primeira e mais crucial etapa do processo foi garantir a
qualidade e a integridade desses dados. Uma descoberta inicial revelou que as informações estavam em uma estrutura aninhada,
exigindo o uso de técnicas específicas, como a normalização de JSON, para transformar os dados em um formato tabular plano e funcional.
A partir daí, um rigoroso processo de limpeza foi aplicado: a coluna de cobranças totais, que continha valores não numéricos para clientes
novos, foi convertida para um formato numérico, e os valores ausentes foram corretamente preenchidos com zero, refletindo a ausência de cobrança
para esses clientes recém-chegados. Para tornar a análise mais intuitiva e acessível, todas as colunas e valores foram traduzidos para o
português. Por fim, todo o conjunto de dados foi convertido para um formato puramente numérico, através de codificação binária e one-hot encoding,
preparando o terreno para uma análise estatística e visual precisa.

Com um conjunto de dados limpo e estruturado, a fase de investigação revelou padrões de comportamento extremamente claros e informativos.
A análise da distribuição geral da evasão já acendeu um alerta: 26,5% dos clientes no dataset haviam cancelado seus serviços, uma taxa expressiva
que valida a urgência desta análise. Ao aprofundar a exploração, um dos fatores de maior impacto emergiu: o tipo de contrato. Clientes com
contratos mensais, que oferecem maior flexibilidade, apresentaram uma taxa de evasão drasticamente superior àqueles com contratos de um ou dois
anos, indicando que o compromisso de longo prazo é um forte fator de retenção. Essa descoberta foi corroborada pela análise do tempo de permanência
do cliente, que mostrou que o risco de evasão é muito maior nos primeiros meses de serviço, diminuindo consideravelmente à medida que o cliente
constrói um histórico mais longo com a empresa. Além disso, o custo do serviço provou ser um ponto sensível; clientes com faturas mensais mais
altas, frequentemente associadas a serviços premium como Fibra Ótica, mostraram uma maior tendência a cancelar. Finalmente, foi observado que a
ausência de serviços de valor agregado, como Suporte Técnico e Segurança Online, era uma característica comum entre os clientes que evadiram,
sugerindo que a percepção de valor e suporte é crucial para a lealdade.

A convergência desses padrões nos permitiu traçar um perfil claro e coeso do cliente com maior risco de evasão: trata-se, tipicamente,
de um cliente recente, com poucos meses de contrato, vinculado a um plano mensal flexível, que paga uma fatura mensal elevada e não possui
serviços de suporte adicionais que poderiam aumentar sua percepção de valor. A combinação de alto custo, baixo compromisso contratual e uma
possível falta de suporte cria o cenário perfeito para o cancelamento.

Com base nessas conclusões concretas, foram formuladas recomendações estratégicas para a Telecom X. Para combater a evasão de clientes com
contratos mensais, sugere-se a criação de ofertas e incentivos agressivos para migrá-los para planos de longo prazo, destacando os benefícios de
custo e estabilidade. Para mitigar o alto risco nos primeiros meses, é recomendável implementar um programa de "onboarding" dedicado aos novos
clientes, com contato proativo e suporte facilitado para garantir uma experiência inicial positiva. Finalmente, para aumentar a retenção entre
os clientes de alto valor e aqueles sem serviços de apoio, a empresa deveria promover ativamente pacotes que incluam Suporte Técnico e Segurança
Online, além de revisar a estrutura de preços dos seus planos premium para garantir que o valor percebido justifique o custo. A implementação dessas
ações, diretamente informadas pela análise de dados, oferece um caminho claro para reduzir a evasão, fortalecer o relacionamento com os clientes e
garantir um crescimento mais sustentável para o negócio.'''